In [19]:
import pandas as pd
import numpy as np
import jellyfish
import re
from nltk.corpus import stopwords

In [2]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [137]:
def cross_product_apply(iteratible, function, symmetric = True):
    length = len(iteratible)
    results_matrix = np.zeros((length,length))
    
    for i in range(0,length):

        if symmetric == False:
            j_length = length
        else:
            j_length = i
            
        for j in range(0,j_length):
            results_matrix[i,j] = function(iteratible[i], iteratible[j])
        
    return results_matrix
    
    
    

In [5]:

def category_matcher(original_categories):
    # want to return n x 2 array of original industries and their matches
    #np.array([[1, 2, 3], [4, 5, 6]], np.int32)
    final_categories = [] #[""] * len(original_categories)
    sim_threshhold = .6
    

    for idx1, original_cat in enumerate(original_categories):

        if len(final_categories) == 0:
            '''
            if it's the first in the list, then a better match can't be found,
            and we keep the first title
            '''
            final_categories.append(original_cat)
        
        else:
            '''
            if it's not the first item, then we compare it to all existing items in the list,
            looking for a match.
            
            if one matches over the threshhold, then we choose that
            otherwise, we choose a new category
            '''
            
            max_sim = 0
            idx_of_max = -1
            
            for idx2, final_category in enumerate(final_categories):                
                sim_rank = similar(original_cat, final_category)

                if sim_rank > max_sim:
                    max_sim = sim_rank
                    idx_of_match = idx2
                        
            if max_sim < sim_threshhold:
                final_categories.append(original_cat)
                #final_categories[idx1] = original_cat
            else:
                final_categories.append(final_categories[idx_of_match])
                #final_categories[idx1] = final_categories[idx_of_match]

    return pd.DataFrame(original_categories,final_categories)

In [87]:
def split_industries(industries): 
    bad_words=list(set(stopwords.words('english'))) + ['']
    splitted_industries = []
    for thing in industries:
        splitted = re.split(pattern='[/|,\s?|&|(?<!-)\s]', string=thing)
        splitted = [word for word in splitted if word not in bad_words]
        splitted_industries.append(splitted)
        
    return splitted_industries



In [108]:
def prop_in_common(iteratible1, iteratible2):
    denominator = len(iteratible1)
    common_count = 0
    if denominator !=0:
        numerator = len(np.intersect1d(iteratible1,iteratible2))
        return 1.0*numerator/denominator
    else:
        return 0.0
    

In [63]:
lobby_city = pd.read_csv('./data/lobbyistFromCity.csv', parse_dates=[0,1])

In [8]:
lobby_time = pd.read_csv('./data/lobbyistTimeRange.csv')

In [9]:
industries = lobby_city.Industry.fillna("").unique()

In [89]:
industries_split = split_industries(industries)

In [138]:
industry_str_sim_matrix = cross_product_apply(industries, jellyfish.jaro_winkler)

In [141]:
industry_list_sim_matrix = cross_product_apply(industries_split, prop_in_common, symmetric=False)

In [184]:
'''
given an n*n matrix, find the number of categories k
such that all n elements are in a category only 
with other elements that have a positive value in the corresponding field
and return those categories
'''

def categorizer(numpy_matrix, threshhold_value = 0):
    length = numpy_matrix.shape[0]
    cluster_candidates = []
    
    for i in range(0,length):
        
        cluster = []
        
        first_vector = numpy_matrix[i,:]
        match_index = first_vector.nonzero()[0]
        
        cluster += [i]
        
        for index in match_index:

            match_value = numpy_matrix[i,index]
            if match_value > threshhold_value:
                cluster += [index]
        
        print(cluster in cluster_candidates)
        cluster_candidates += set(cluster)
        #print(industries[cluster])
    
    return(len(set(cluster_candidates)))
    
    
    
#clusters = 
categorizer(industry_list_sim_matrix)

False


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [179]:
'''
Given these candidates, what will happen if we choose to minimize the number of single-element groups
'''
def choose_best_clusters(clusters):
    elements = set([x for x in clusters])

    


choose_best_clusters(clusters)

322

In [145]:
for i in range(0,30):
    vector = industry_str_sim_matrix[i,:]

    print(industries[i] + " : " + str(industries[(vector > .75).nonzero()]))

Land Development & Sales : []
Real Estate Investment/Development : []
Real Estate Investment : ['Real Estate Investment/Development']
Public School : []
Real Estate Development : ['Real Estate Investment/Development' 'Real Estate Investment']
Real Estate : ['Real Estate Investment/Development' 'Real Estate Investment'
 'Real Estate Development']
Religious Assembly/Education : []
Restaurant Association : []
Real Estate Investment/ Development : ['Real Estate Investment/Development' 'Real Estate Investment'
 'Real Estate Development' 'Real Estate']
Sports and Special Events Productions : []
Education : []
Hotel : []
Religious Assembly : ['Religious Assembly/Education']
Private Property Owner : []
Municipal Utility District : []
Ranching : []
School : []
Restaurant : ['Real Estate' 'Restaurant Association']
University : []
Provides internet related products and services : []
Real Estate/Investment : ['Real Estate Investment/Development' 'Real Estate Investment'
 'Real Estate Development' 

# We now have two representations of similarity between elements

# The next step is to cluster by those similarities 

In [8]:
lob_cli = lobby_city.groupby(['Lobbyist','Client'])

NameError: name 'lobby_city' is not defined

In [51]:
lob_ind = lobby_city.groupby(['Lobbyist','Industry'])

In [57]:
len(array_of_industries)

322

In [23]:
lobby_data.head(20)

,Start,End,Lobbyist,Client,Industry
0,2013-12-20,2014-12-20,Richard Suttle,Forestar (USA) Real Estate Group,Land Development & Sales
1,2013-12-20,2014-12-20,Richard Suttle,Tierra Concepts,Real Estate Investment/Development
2,2013-12-20,2014-12-20,Richard Suttle,Majestic Realty,Real Estate Investment
3,2013-12-20,2014-12-20,Richard Suttle,"Cullen, John",Real Estate Investment
4,2013-12-20,2014-12-20,Richard Suttle,"The NRP Group, LLC",Real Estate Investment
5,2013-12-20,2014-12-20,Richard Suttle,The Charter School Fund,Public School
6,2013-12-20,2014-12-20,Richard Suttle,"Rice, Robert",Real Estate Investment
7,2013-12-20,2014-12-20,Richard Suttle,"Wilkerson, Eric",Real Estate Investment
8,2013-12-20,2014-12-20,Richard Suttle,Riddell Family Limited Partnership,Real Estate Investment
9,2013-12-20,2014-12-20,Richard Suttle,Autoreturn,Real Estate Investment
